In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | model

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [7]:
with_message_history.invoke(
    {"ability": "math", "input": "What does cosine mean?"},
    config={"configurable": {"session_id": "abc123"}},
)

Parent run ab6add3b-1855-4b97-87ab-47114ebc7c83 not found for run f88806a5-10fb-4b5a-96da-c15cdbc9503e. Treating as a root run.


AIMessage(content='Cosine is a trigonometric function that gives the ratio of the adjacent side to the hypotenuse in a right triangle.', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 33, 'total_tokens': 59}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0d834d65-728c-4845-b87c-26dd426a4308-0')

프롬프트 변수명 input에서 다른 것으로 변경

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)
runnable = prompt | model

In [9]:
with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="message",
    history_messages_key="history",
)

In [10]:
with_message_history.invoke(
    {"ability": "math", "message": "코싸인이 뭐야?"},
    config={"configurable": {"session_id": "demo-240517"}},
)

Parent run 64981644-df20-4cde-a8f0-e39383693fc3 not found for run a89599e9-4e63-4d9c-9c7d-4ed61dfa52b2. Treating as a root run.


AIMessage(content='코싸인은 삼각함수 중 하나로, 직각삼각형의 빗변을 밑변으로 나눈 값입니다.', response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 38, 'total_tokens': 85}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5b468b4b-1a44-45de-9f7f-cf1c4aaa1006-0')

In [12]:
store["demo-240517"].messages

[HumanMessage(content='코싸인이 뭐야?'),
 AIMessage(content='코싸인은 삼각함수 중 하나로, 직각삼각형의 빗변을 밑변으로 나눈 값입니다.', response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 38, 'total_tokens': 85}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5b468b4b-1a44-45de-9f7f-cf1c4aaa1006-0')]

In [13]:
with_message_history.invoke(
    {"ability": "math", "message": "뭐라고?"},
    config={"configurable": {"session_id": "demo-240517"}},
)

Parent run b0cc46e0-5813-4b88-a306-06c35fe1c5ec not found for run 6c99aa9f-ad24-4bce-b919-0e1564219693. Treating as a root run.


AIMessage(content='코싸인은 사인의 역수입니다. 즉, 코싸인 θ = 1 / 사인 θ 입니다.', response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 98, 'total_tokens': 134}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d9f2ec73-4e27-4259-a4f7-4af16d463afe-0')

In [14]:
store["demo-240517"].messages

[HumanMessage(content='코싸인이 뭐야?'),
 AIMessage(content='코싸인은 삼각함수 중 하나로, 직각삼각형의 빗변을 밑변으로 나눈 값입니다.', response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 38, 'total_tokens': 85}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5b468b4b-1a44-45de-9f7f-cf1c4aaa1006-0'),
 HumanMessage(content='뭐라고?'),
 AIMessage(content='코싸인은 사인의 역수입니다. 즉, 코싸인 θ = 1 / 사인 θ 입니다.', response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 98, 'total_tokens': 134}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d9f2ec73-4e27-4259-a4f7-4af16d463afe-0')]

# RunnableLambda 방법 시도

In [25]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [26]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

dummy_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. If order_id is provided, you need to output '조회'.
            If order_id is not provided, you need to classify the customer input message into one of the following two types:
            - 상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            - 주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)

In [27]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [28]:
dummy_chain = dummy_prompt | model 

In [30]:
from langchain_core.runnables.history import RunnableWithMessageHistory

dumy_chain_with_memory = RunnableWithMessageHistory(
    dummy_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [31]:
store = {}

In [32]:
response = dumy_chain_with_memory.invoke(
    {"input": "주문 취소하고 싶어", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240516-1"}}
    )
response

Parent run aad6332f-9f70-48d0-b70f-45750f4b2b91 not found for run 5715a27a-3c8e-4522-82c3-5f394135f0f7. Treating as a root run.


AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cd6fc415-7c3e-4471-9451-6475740c49c8-0')

In [34]:
store["test_240516-1"].messages

[HumanMessage(content='주문 취소하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cd6fc415-7c3e-4471-9451-6475740c49c8-0')]

In [35]:
from langchain_core.runnables import RunnableLambda

exp = RunnableLambda(
    lambda x: dumy_chain_with_memory.invoke(x, config={"configurable": {"session_id": "test_240516-1"}})
    )
exp

RunnableLambda(lambda x: dumy_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240516-1'}}))

In [36]:
exp.invoke(
    {"input": "판매 상품 알고 싶어", 
    "order_id": None}
)

Parent run 1192318f-91e5-4875-83ff-ba8aa9c52662 not found for run 5d25b193-b3ed-479b-bd59-98f631f7de46. Treating as a root run.


AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 175, 'total_tokens': 177}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-eff88246-14cd-4e68-96fc-d927faae5da4-0')

In [37]:
store["test_240516-1"].messages

[HumanMessage(content='주문 취소하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cd6fc415-7c3e-4471-9451-6475740c49c8-0'),
 HumanMessage(content='판매 상품 알고 싶어'),
 AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 175, 'total_tokens': 177}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-eff88246-14cd-4e68-96fc-d927faae5da4-0')]

In [38]:
exp.invoke(
    {"input": None, 
    "order_id": None}
)

Parent run 3939c880-4945-4b25-847a-b10620ac87d7 not found for run 9ced51eb-79fc-4c8d-9597-f33f689856fd. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got None.')


AIMessage(content='Please provide more information.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 186, 'total_tokens': 191}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d695fd47-9bf6-43b7-a476-f65258a83432-0')

In [39]:
store["test_240516-1"].messages

[HumanMessage(content='주문 취소하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cd6fc415-7c3e-4471-9451-6475740c49c8-0'),
 HumanMessage(content='판매 상품 알고 싶어'),
 AIMessage(content='문의', response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 175, 'total_tokens': 177}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-eff88246-14cd-4e68-96fc-d927faae5da4-0')]